In [1]:
import geopandas as gpd
import pandas as pd
import zipfile
import requests
import os
import simplekml
import folium
from shapely.geometry import MultiPolygon
from streamlit_folium import st_folium
import streamlit as st

# Function to automatically download GADM shapefiles for any country code
def download_and_extract_gadm_shapefiles(country_code, output_path):
    url = f"https://geodata.ucdavis.edu/gadm/gadm4.1/shp/gadm41_{country_code}_shp.zip"
    zip_file_path = os.path.join(output_path, f"gadm41_{country_code}_shp.zip")

    response = requests.get(url)
    if response.status_code == 200:
        with open(zip_file_path, 'wb') as f:
            f.write(response.content)

        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(output_path)

        os.remove(zip_file_path)
        print(f"✅ Downloaded and extracted shapefiles for {country_code}.")
    else:
        print(f"❌ Failed to download shapefiles for {country_code}.")
        return None

# Path to save and extract the downloaded GADM shapefiles
base_path = r"C:\\Users\\X1 Carbon\\Desktop\\gadm_shapefiles"  # Updated path to your downloaded folder
os.makedirs(base_path, exist_ok=True)

# Streamlit input for country code (e.g., 'DEU' for Germany, 'PAK' for Pakistan)
country_code = st.text_input("Enter Country Code (e.g., 'DEU' for Germany, 'PAK' for Pakistan):", "PAK").strip()

# Download and extract the GADM shapefiles for the given country code
if not os.path.exists(os.path.join(base_path, f"gadm41_{country_code}_0.shp")):
    download_and_extract_gadm_shapefiles(country_code, base_path)

# Load shapefiles for all administrative levels (skipping missing ones)
level_files = {
    0: os.path.join(base_path, f"gadm41_{country_code}_0.shp"),
    1: os.path.join(base_path, f"gadm41_{country_code}_1.shp"),
    2: os.path.join(base_path, f"gadm41_{country_code}_2.shp"),
    3: os.path.join(base_path, f"gadm41_{country_code}_3.shp"),
    4: os.path.join(base_path, f"gadm41_{country_code}_4.shp"),
}

all_levels = []
for level, path in level_files.items():
    if os.path.exists(path):
        gdf = gpd.read_file(path)
        gdf["level"] = level
        all_levels.append(gdf)
    else:
        print(f"⚠️ Missing: {path}")

# Combine all levels into a single GeoDataFrame (excluding missing levels)
if len(all_levels) > 0:
    full_gdf = gpd.GeoDataFrame(pd.concat(all_levels, ignore_index=True))
    print(f"✅ Loaded all levels: {full_gdf.shape}")
else:
    print("❌ No shapefiles were found for the specified country code.")

# Export helper for KML
def export_to_kml(gdf, filepath):
    kml = simplekml.Kml()
    for _, row in gdf.iterrows():
        geom = row.geometry
        if geom.geom_type == "Polygon":
            coords = [(x, y) for x, y in geom.exterior.coords]
            pol = kml.newpolygon(name="Area", outerboundaryis=coords)
        elif geom.geom_type == "MultiPolygon":
            for part in geom:
                coords = [(x, y) for x, y in part.exterior.coords]
                pol = kml.newpolygon(name="Area", outerboundaryis=coords)
    kml.save(filepath)

# Streamlit Widgets for user input
query = st.text_input("🔍 Enter Province/City Name")
format_dropdown = st.selectbox("💾 Export Format", ["geojson", "shp", "kml"])
combine_box = st.checkbox("🔗 Combine Multiple Matches into One Shape")
run_button = st.button("Search & Export")

# Main export/display logic
def export_and_show(gdf, q, fmt, combine):
    gdf_projected = gdf.to_crs(epsg=32632)  # Projecting to UTM for accurate area/centroid

    # Compute centroid in the projected CRS
    centroid = gdf_projected.geometry.centroid.iloc[0]
    lon, lat = centroid.x, centroid.y
    area_km2 = gdf_projected.geometry.area.iloc[0] / 1e6

    # Convert projected coordinates (UTM) to WGS84 for proper lat, lon
    gdf_wgs84 = gdf_projected.to_crs(epsg=4326)  # Convert to WGS84 (lat, lon)
    centroid_wgs84 = gdf_wgs84.geometry.centroid.iloc[0]
    lat_wgs84, lon_wgs84 = centroid_wgs84.y, centroid_wgs84.x

    # Show map
    centroid_original = gdf.geometry.centroid.iloc[0]
    lat_display, lon_display = centroid_original.y, centroid_original.x
    m = folium.Map(location=[lat_display, lon_display], zoom_start=8)
    folium.GeoJson(gdf).add_to(m)
    m.save("temp_map.html")
    st.components.v1.html(open("temp_map.html", "r").read(), height=500)

    # Display metrics
    st.metric("📐 Area (km²)", f"{area_km2:.2f}")
    st.metric("📍 Centroid", f"Lat: {lat_wgs84:.4f}, Lon: {lon_wgs84:.4f}")

    # Export logic (GeoJSON, SHP, KML)
    output_folder = r"C:\Users\X1 Carbon\Desktop\exports"
    os.makedirs(output_folder, exist_ok=True)
    suffix = "combined" if combine else "selected"
    path = f"{output_folder}\\{q.replace(' ', '_')}_{suffix}.{fmt}"

    if fmt == "geojson":
        gdf.to_file(path, driver="GeoJSON")
    elif fmt == "shp":
        gdf.to_file(path)
    elif fmt == "kml":
        export_to_kml(gdf, path)

    st.download_button("📥 Download Exported File", path)

# Button click handler for Streamlit
if run_button:
    q = query.strip().lower()
    fmt = format_dropdown
    combine = combine_box

    if not q:
        st.warning("❗ Please enter a name to search.")
    else:
        name_cols = [col for col in full_gdf.columns if col.startswith("NAME_")]
        matches = full_gdf[
            full_gdf[name_cols].apply(lambda row: any(q in str(val).lower() for val in row if pd.notnull(val)), axis=1)
        ]

        if matches.empty:
            st.warning("❌ No match found.")
        else:
            st.success(f"✅ Found {len(matches)} matches.")

            if combine:
                selected = matches.dissolve()
                export_and_show(selected, q, fmt, combine)
            else:
                options = [
                    (" > ".join([str(row[c]) for c in name_cols if pd.notnull(row[c])]), i)
                    for i, row in matches.iterrows()
                ]
                labels = [label for label, idx in options]
                selected_label = st.selectbox("🎯 Select one match:", labels)
                selected_idx = dict(options)[selected_label]
                selected = gpd.GeoDataFrame([matches.loc[selected_idx]], crs=matches.crs)

                export_and_show(selected, q, fmt, combine)


2025-08-10 15:19:50.122 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-08-10 15:19:50.499 
  command:

    streamlit run C:\Users\X1 Carbon\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]


⚠️ Missing: C:\\Users\\X1 Carbon\\Desktop\\gadm_shapefiles\gadm41_PAK_4.shp
✅ Loaded all levels: (183, 29)
